In [1]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 
import os,glob

In [3]:
# 관광지명, 도로명주소, lat, lng, 분류
df = pd.read_excel('data/서울시 관광 쇼핑_ljh.XLSX', engine='openpyxl')
df.head(2)

,고유번호,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,교통정보
0,64,ja,トンデムン(東大門)ショッピングタウン,100-951 ソウル市中?乙支路6街,04564 ソウル市チュン(中)?チャンチュンダンロ253 (乙支路6街),-40,http://www.ddm-mall.co.kr/,店?により異なる,地下?4?線「トンデムン·ヨッサムンファゴンウォン」?1番、14番出口からすぐ 地下?4?線...
1,193,ja,トンデムン(東大門)チャンシンドン(昌信洞)玩具??り市場,110-841 ソウル市鐘路?昌信洞 390-29,03120 ソウル市チョンノ(鐘路)?チョンロ52ギル36 (昌信洞),-40,NaN,店?により異なる,地下?1?線「トンデムン」?4番出口徒?5分 地下?6?線「トンミョアプ」?6番出口徒?5分


In [4]:
df.to_csv('data/main/temp_files/서울_쇼핑.csv', index=False)

In [8]:
df = pd.read_csv('data/main/temp_files/서울_쇼핑.csv', encoding='utf-8', sep=',')
df.head(2)

,고유번호,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,교통정보
0,64,ja,トンデムン(東大門)ショッピングタウン,100-951 ソウル市中?乙支路6街,04564 ソウル市チュン(中)?チャンチュンダンロ253 (乙支路6街),-40,http://www.ddm-mall.co.kr/,店?により異なる,地下?4?線「トンデムン·ヨッサムンファゴンウォン」?1番、14番出口からすぐ 地下?4?線...
1,193,ja,トンデムン(東大門)チャンシンドン(昌信洞)玩具??り市場,110-841 ソウル市鐘路?昌信洞 390-29,03120 ソウル市チョンノ(鐘路)?チョンロ52ギル36 (昌信洞),-40,NaN,店?により異なる,地下?1?線「トンデムン」?4番出口徒?5分 地下?6?線「トンミョアプ」?6番出口徒?5分


In [9]:
df = df[(df['언어']) == 'ko']
df.head(4)

,고유번호,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,교통정보
4,281,ko,통인시장,110-043 서울 종로구 통인동 10-3,03036 서울 종로구 자하문로15길 18,02-722-0911,http://tonginmarket.co.kr,"통인시장 영업시간 : 7:00 ~ 21:00 ※ 상점에 따라 상이하며, 정확한 개점...",3호선 경복궁역 2번 출구 → 2번 출구와 3번 출구 사이 도로를 따라 직진
5,291,ko,신사동 가로수길,135-888 서울 강남구 신사동 530 일대,06028 서울 강남구 압구정로 120 일대,NaN,NaN,NaN,3호선 신사역 8번 출구 도보 10분
9,1272,ko,풍월당,"135-897 서울 강남구 신사동 657-37 성산빌딩 4, 5층","06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)",02-512-2222,http://www.pungwoldang.kr/,12:00 ~ 20:00 (KST),3호선 압구정역 2번 출구 버스환승 7호선 강남구청역 4번 출구 버스환승 ...
11,1770,ko,화개길 구두거리,서울 종로구 화동 화개길,서울 종로구 화동화개길,02-120,NaN,매장별로 차이가 있음,3호선 경복궁 5번 출구에서 약 1.5 킬로미터 3호선 안국 1번 출구에서 약 1킬...


In [4]:

df.언어.unique()

array(['ko'], dtype=object)

In [10]:
## 도로명주소 불안정 -> 주소도 같이 남김
df.rename(columns= {'상호명':'관광지명', '신주소':'도로명주소'}, inplace=True)
df = df[['관광지명', '도로명주소']]
df.reset_index(drop=True,inplace=True)
df.head()

,관광지명,도로명주소
0,통인시장,03036 서울 종로구 자하문로15길 18
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)"
3,화개길 구두거리,서울 종로구 화동화개길
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4)


In [11]:
df.nunique()

관광지명     162
도로명주소    160
dtype: int64

In [12]:
df.to_csv('data/서울_쇼핑.csv',index=False)

In [13]:
df1= pd.read_csv('data/서울_쇼핑.csv', encoding='utf-8', sep=',')
df1

,관광지명,도로명주소
0,통인시장,03036 서울 종로구 자하문로15길 18
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)"
3,화개길 구두거리,서울 종로구 화동화개길
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4)
...,...,...
158,스타일난다 핑크 호텔 (명동 플래그십 스토어),04536 서울 중구 명동8길 37-8 (충무로2가) 스타일난다 핑크호텔
159,아모레 플래그십 스토어 (성수),"04797 서울 성동구 아차산로11길 7 (성수동2가, (주)아세아이엔티)"
160,경리단길,04344 서울 용산구 회나무로 일대(이태원동)
161,명동문고 명동성당점,04537 서울 중구 명동길 66 (명동2가) 명동성당 지하 1층


In [14]:
df1.head()

,관광지명,도로명주소
0,통인시장,03036 서울 종로구 자하문로15길 18
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)"
3,화개길 구두거리,서울 종로구 화동화개길
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4)


In [12]:
# df1['도로명주소'].str.replace('(', '',regex=False)
# df1['도로명주소']

In [13]:
# df1['주소'].str.replace('(', '',regex=False)
# df1['주소'][51:100]     # ( ), :, / --> 모두 없애기??

In [14]:
# df1 = df1.to_csv('data/서울_명소_주소.csv', index=False)

In [15]:
####################################################################
# 위까지는 전처리
# 아래부터는 도로명주소 기반으로 좌표 구해서 추가함
####################################################################
# df2 = pd.read_csv('data/서울_명소_쇼핑.csv', encoding='utf-8', sep=',')
# df2.head(3)

,관광지명,도로명주소,주소
0,통인시장,03036 서울 종로구 자하문로15길 18,110-043 서울 종로구 통인동 10-3
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대,135-888 서울 강남구 신사동 530 일대
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)","135-897 서울 강남구 신사동 657-37 성산빌딩 4, 5층"


In [16]:
# df1.drop(columns=['주소'], inplace=True)
# df1.head()

,관광지명,도로명주소
0,통인시장,03036 서울 종로구 자하문로15길 18
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)"
3,화개길 구두거리,서울 종로구 화동화개길
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4)


In [15]:
df1['관광지명'][1][:-1]

'신사동 가로수'

``` test ```

In [1]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 
import os,glob

In [2]:
df= pd.read_csv('data/서울_쇼핑.csv', encoding='utf-8', sep=',')
df

,관광지명,도로명주소
0,통인시장,03036 서울 종로구 자하문로15길 18
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)"
3,화개길 구두거리,서울 종로구 화동화개길
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4)
...,...,...
158,스타일난다 핑크 호텔 (명동 플래그십 스토어),04536 서울 중구 명동8길 37-8 (충무로2가) 스타일난다 핑크호텔
159,아모레 플래그십 스토어 (성수),"04797 서울 성동구 아차산로11길 7 (성수동2가, (주)아세아이엔티)"
160,경리단길,04344 서울 용산구 회나무로 일대(이태원동)
161,명동문고 명동성당점,04537 서울 중구 명동길 66 (명동2가) 명동성당 지하 1층


In [29]:
# 2 kakao API로 도로명 주소 구하는 함수
def kakao_location(place):
    with open('data/main/key/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [39]:
# 2 입력받은 데이터 프레임의 인덱스로 도로명주소 추출하는 함수
def rtn_addr(target):
    str_addr = df[df.관광지명 == target].도로명주소.values
    # str_addr = df.도로명주소.values[i].strip()
    return str_addr

In [23]:
# type(df.도로명주소[0])
df.도로명주소.values[1].strip()

'06028 서울 강남구 압구정로 120 일대'

In [38]:
target = df['관광지명'][0].strip()    # '통인시장'
df[df.관광지명 == target].df['도로명주소'].values # array
# df[df.관광지명 == '통인시장'].도로명주소.values # array

AttributeError: 'DataFrame' object has no attribute 'df'

In [40]:
# 3 2번 함수들 사용해서 좌표추가
temp1 =[]
for i in df.index:
    try:
        target = df['관광지명'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
        # load_val = df.도로명주소.values[i].strip()
        # temp1.append(kakao_location(load_val))
    except:
        print(i, df.관광지명[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df_addr = pd.concat([df, df_test], axis=1)

0 통인시장
1 신사동 가로수길
2 풍월당
3 화개길 구두거리
4 테크노마트
5 콴챈루거리
6 목동 로데오거리
7 동묘 벼룩시장
8 서촌구루루
9 자비시주얼리
10 카카오프렌즈 강남 플래그십 스토어
11 한신옹기
12 서울 시내 핵심 관광지 ‘자유이용권’ <디스커버서울패스>
13 쌈지길
14 노앙(NOHANT)
15 아로마티카
16 엔터식스 패션스트릿 강남점
17 명동 쇼핑
18 인사동길
19 타임스퀘어
20 금천 관광 추천코스
21 종각지하쇼핑센터
22 종로 귀금속 거리
23 방학동 도깨비시장
24 센트럴시티
25 에비수
26 밀리오레 명동점
27 강남역 지하쇼핑센터
28 에이바이봄
29 젠틀몬스터 신사 플래그쉽 스토어
30 고투몰
31 증산종합시장 
32 송리단길
33 제로띵스
34 광장시장
35 SMTOWN&STORE@DDP
36 홍대앞 예술시장 프리마켓
37 서울풍물시장 
38 현대백화점 본점
39 아티지 (ARTAGE)
40 레어마켓 (Rare Market)
41 마이시티 상암점
42 송림수제화
43 연서시장
44 알맹상점
45 망원시장
46 젠틀몬스터 홍대 플래그십스토어
47 설화수 플래그십 스토어
48 망리단길
49 청량리 전통시장
50 청량리 통닭골목
51 신평화 패션타운
52 롯데백화점 본점
53 코엑스몰
54 두타(DOOTA)
55 IFC몰
56 제니하우스
57 삼성 딜라이트
58 서울약령시장
59 문정동 로데오거리
60 숭례문 수입상가
61 용산전자상가
62 명동지하쇼핑센터
63 에뛰드하우스
64 롯데 영플라자 명동점
65 Re:CODE 
66 10 꼬르소 꼬모
67 정샘물 인스피레이션
68 용산전자랜드
69 한국관광명품점
70 가락몰
71 성수동 수제화 거리
72 더 피커
73 해방촌 신흥시장
74 쌍리단길
75 RECTO(렉토)
76 남대문시장
77 이대 패션거리
78 교보 핫트랙스 광화문점
79 청계5가 지하쇼핑센터
80 소공 지하쇼핑센터
81 청담 패션 거리 
82 동대문 헌책방 거리
83 갤러리아백화점 WEST
84 뚝섬 아름다운 

In [42]:
df_addr.head(10)

,관광지명,도로명주소,lat,lng
0,통인시장,03036 서울 종로구 자하문로15길 18,NaN,NaN
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대,NaN,NaN
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)",NaN,NaN
3,화개길 구두거리,서울 종로구 화동화개길,NaN,NaN
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4),NaN,NaN
5,콴챈루거리,04536 서울시 중구 명동2길 27 (명동2가) 일대,NaN,NaN
6,목동 로데오거리,서울특별시 양천구 목동로25길,NaN,NaN
7,동묘 벼룩시장,03116 서울 종로구 난계로 243 (숭인동),NaN,NaN
8,서촌구루루,03040 서울 종로구 필운대로 26 (누하동),NaN,NaN
9,자비시주얼리,"06017 서울 강남구 압구정로42길 36 (신사동, 금광아크존1) 1층",NaN,NaN


In [67]:
# 3 2번 함수들 사용해서 좌표추가
temp1 =[]
for i in df1.index:
    try:
        target = df1['관광지명'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
        # temp1.append(kakao_location(df.도로명주소.values[i]))
    except:
        print(i, df1.관광지명[i])
        
df1_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df1, df1_test], axis=1)
df2.head()

0 통인시장
1 신사동 가로수길
2 풍월당
3 화개길 구두거리
4 테크노마트
5 콴챈루거리
6 목동 로데오거리
7 동묘 벼룩시장
8 서촌구루루
9 자비시주얼리
10 카카오프렌즈 강남 플래그십 스토어
11 한신옹기
12 서울 시내 핵심 관광지 ‘자유이용권’ <디스커버서울패스>
13 쌈지길
14 노앙(NOHANT)
15 아로마티카
16 엔터식스 패션스트릿 강남점
17 명동 쇼핑
18 인사동길
19 타임스퀘어
20 금천 관광 추천코스
21 종각지하쇼핑센터
22 종로 귀금속 거리
23 방학동 도깨비시장
24 센트럴시티
25 에비수
26 밀리오레 명동점
27 강남역 지하쇼핑센터
28 에이바이봄
29 젠틀몬스터 신사 플래그쉽 스토어
30 고투몰
31 증산종합시장 
32 송리단길
33 제로띵스
34 광장시장
35 SMTOWN&STORE@DDP
36 홍대앞 예술시장 프리마켓
37 서울풍물시장 
38 현대백화점 본점
39 아티지 (ARTAGE)
40 레어마켓 (Rare Market)
41 마이시티 상암점
42 송림수제화
43 연서시장
44 알맹상점
45 망원시장
46 젠틀몬스터 홍대 플래그십스토어
47 설화수 플래그십 스토어
48 망리단길
49 청량리 전통시장
50 청량리 통닭골목
51 신평화 패션타운
52 롯데백화점 본점
53 코엑스몰
54 두타(DOOTA)
55 IFC몰
56 제니하우스
57 삼성 딜라이트
58 서울약령시장
59 문정동 로데오거리
60 숭례문 수입상가
61 용산전자상가
62 명동지하쇼핑센터
63 에뛰드하우스
64 롯데 영플라자 명동점
65 Re:CODE 
66 10 꼬르소 꼬모
67 정샘물 인스피레이션
68 용산전자랜드
69 한국관광명품점
70 가락몰
71 성수동 수제화 거리
72 더 피커
73 해방촌 신흥시장
74 쌍리단길
75 RECTO(렉토)
76 남대문시장
77 이대 패션거리
78 교보 핫트랙스 광화문점
79 청계5가 지하쇼핑센터
80 소공 지하쇼핑센터
81 청담 패션 거리 
82 동대문 헌책방 거리
83 갤러리아백화점 WEST
84 뚝섬 아름다운 

,관광지명,도로명주소,lat,lng
0,통인시장,03036 서울 종로구 자하문로15길 18,NaN,NaN
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대,NaN,NaN
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)",NaN,NaN
3,화개길 구두거리,서울 종로구 화동화개길,NaN,NaN
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4),NaN,NaN


In [62]:
# 검증
temp = df2['관광지명'][0]
rtn_addr(temp)

AttributeError: 'numpy.ndarray' object has no attribute 'strip'

In [58]:
df2.shape, df2.head(10)

((163, 4),
        관광지명                                      도로명주소        lat         lng
 0      통인시장                   03036 서울 종로구 자하문로15길 18   37.528055  126.861763
 1  신사동 가로수길                   06028 서울 강남구 압구정로 120 일대  37.568735  127.007666
 2       풍월당  06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)   37.528475  127.040053
 3  화개길 구두거리                              서울 종로구 화동화개길   37.532071  126.994591
 4     테크노마트       05116 서울 광진구 광나루로56길 85 (구의동 546-4)   37.503935  127.002413
 5     콴챈루거리           04536 서울시 중구 명동2길 27  (명동2가) 일대   37.564205  126.981638
 6  목동 로데오거리                          서울특별시 양천구 목동로25길         NaN         NaN
 7   동묘 벼룩시장                03116 서울 종로구 난계로 243 (숭인동)         NaN         NaN
 8     서촌구루루                03040 서울 종로구 필운대로 26 (누하동)         NaN         NaN
 9    자비시주얼리   06017 서울 강남구 압구정로42길 36 (신사동, 금광아크존1) 1층        NaN         NaN)

In [31]:
# 4 처리 끝난 파일 저장
df2.to_csv('data/쇼핑_addr.csv',index=False)

In [92]:
df = pd.read_csv('data/쇼핑_addr.csv')
df.head(10)

,관광지명,도로명주소,lat,lng
0,통인시장,03036 서울 종로구 자하문로15길 18,37.528055,126.861763
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대,37.568735,127.007666
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)",37.528475,127.040053
3,화개길 구두거리,서울 종로구 화동화개길,37.532071,126.994591
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4),37.503935,127.002413
5,콴챈루거리,04536 서울시 중구 명동2길 27 (명동2가) 일대,37.564205,126.981638
6,목동 로데오거리,서울특별시 양천구 목동로25길,NaN,NaN
7,동묘 벼룩시장,03116 서울 종로구 난계로 243 (숭인동),NaN,NaN
8,서촌구루루,03040 서울 종로구 필운대로 26 (누하동),NaN,NaN
9,자비시주얼리,"06017 서울 강남구 압구정로42길 36 (신사동, 금광아크존1) 1층",NaN,NaN


In [85]:
type(df['도로명주소'].iloc[0][0:5])

str

In [89]:
df['도로명주소'].str.split().str[0]

0      03036
1      06028
2      06018
3         서울
4      05116
       ...  
158    04536
159    04797
160    04344
161    04537
162    06545
Name: 도로명주소, Length: 163, dtype: object

In [91]:
df['도로명주소'].iloc[3][0]

' '

In [ ]:
# 우편번호 없애기
# df['도로명주소']= df['도로명주소'].str.split().str[1:]
ad_list = [] 
for i in df.index:
    df['도로명주소'][i] = df['도로명주소'][i][1:]
# df.drop_duplicates(subset=['도로명주소'],keep='first',inplace=True)
    ad_list.append(df['도로명주소'][i])
ad_list
# df.head()

In [52]:
df.도로명주소

0      []
1      []
2      []
3      []
4      []
       ..
158    []
159    []
160    []
161    []
162    []
Name: 도로명주소, Length: 163, dtype: object

In [93]:
kakao_location('서울특별시 양천구 목동로25길	')

(37.5280548976635, 126.861763072861)

In [34]:
df.to_csv('data/쇼핑_addr_final.csv', index=False)

In [35]:
df_test = pd.read_csv('data/쇼핑_addr_final.csv', encoding='utf-8', sep=',')
df_test.head()

,관광지명,도로명주소,lat,lng
0,통인시장,03036 서울 종로구 자하문로15길 18,37.528055,126.861763
1,신사동 가로수길,06028 서울 강남구 압구정로 120 일대,37.568735,127.007666
2,풍월당,"06018 서울 강남구 도산대로53길 39 (신사동, 신사동 C/K빌딩)",37.528475,127.040053
3,화개길 구두거리,서울 종로구 화동화개길,37.532071,126.994591
4,테크노마트,05116 서울 광진구 광나루로56길 85 (구의동 546-4),37.503935,127.002413
